In [14]:
#common
import pandas as pd
import numpy as np
import math
import random

# metrics
from behalearn import metrics

#scikit-learn
from sklearn.model_selection import StratifiedKFold
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import VarianceThreshold

In [15]:
data_path = './data/'
extracted_features = 'extracted_features.csv'

In [16]:
df = pd.read_csv(data_path + extracted_features)
df.head(10)

,user_id,doc_id,interstroke_time,stroke_duration,start_x,start_y,stop_x,stop_y,direct_end_to_end_distance,mean_resultant_lenght,...,average_direction,length_of_trajectory,ratio_end_to_end_dist_and_length_of_trajectory,average_velocity,median_acceleration_at_first_5_points,midstroke_pressure,midstroke_area_covered,midstroke_finger_orientation,change_of_finger_orientation,phone_orientation
0,37,4,0.341,0.256,27.416,-27.113,12.3980,-32.960,16.1160,0.35736,...,-2.581200,16.1320,0.99898,63.018,-432.24,0.320,0.044444,0.0,0,1
1,37,4,0.913,0.086,21.771,-30.037,9.8778,-33.867,12.4950,0.26504,...,-2.199400,12.5960,0.99197,146.470,5117.70,0.600,0.088889,0.0,0,1
2,37,4,0.733,0.111,10.886,-34.169,26.9120,-34.673,16.0340,0.99118,...,0.086288,16.2030,0.98959,145.970,-3563.50,0.480,0.044444,0.0,0,1
3,37,4,0.448,0.094,19.151,-34.572,12.7000,-36.790,6.8213,0.59647,...,-2.723100,6.9674,0.97902,74.122,-4604.10,0.400,0.044444,0.0,0,1
4,37,4,0.519,0.056,27.718,-32.758,14.6150,-37.294,13.8660,0.67819,...,-2.717000,13.8710,0.99965,247.690,-21377.00,0.440,0.044444,0.0,0,1
5,37,4,0.290,0.115,30.137,-30.641,15.8250,-35.479,15.1080,0.79869,...,-2.791400,15.2220,0.99254,132.360,2831.10,0.405,0.044444,0.0,0,1
6,37,4,1.001,0.091,22.376,-32.355,11.3900,-33.967,11.1040,0.74862,...,-2.956600,11.1380,0.99700,122.390,-4931.10,0.500,0.044444,0.0,0,1
7,37,4,0.287,0.102,31.750,-31.044,20.7630,-36.286,12.1730,0.57991,...,-2.540400,12.2440,0.99418,120.040,-5321.50,0.390,0.044444,0.0,0,1
8,37,4,1.627,0.103,22.679,-34.270,10.6840,-35.379,12.0460,0.55388,...,-2.985900,12.1000,0.99547,117.480,-2638.20,0.470,0.044444,0.0,0,1
9,37,4,0.220,0.045,12.095,-31.952,16.4290,-32.456,4.3633,0.99472,...,-0.033020,4.3996,0.99175,97.770,-14737.00,0.380,0.044444,0.0,0,1


In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21158 entries, 0 to 21157
Data columns (total 34 columns):
user_id                                           21158 non-null int64
doc_id                                            21158 non-null int64
interstroke_time                                  21119 non-null float64
stroke_duration                                   21158 non-null float64
start_x                                           21158 non-null float64
start_y                                           21158 non-null float64
stop_x                                            21158 non-null float64
stop_y                                            21158 non-null float64
direct_end_to_end_distance                        21158 non-null float64
mean_resultant_lenght                             21158 non-null float64
orientation_flag                                  21158 non-null int64
direction_of_end_to_end_line                      21158 non-null float64
phone_id             

In [22]:
df_descriptions = df.describe()
df_descriptions

,user_id,interstroke_time,stroke_duration,start_x,start_y,stop_x,stop_y,direct_end_to_end_distance,mean_resultant_lenght,orientation_flag,...,average_direction,length_of_trajectory,ratio_end_to_end_dist_and_length_of_trajectory,average_velocity,median_acceleration_at_first_5_points,midstroke_pressure,midstroke_area_covered,midstroke_finger_orientation,change_of_finger_orientation,phone_orientation
count,20848.000000,2.084800e+04,20848.000000,20848.000000,20848.000000,20848.000000,20848.000000,20848.000000,2.084800e+04,20848.000000,...,20848.000000,20848.000000,20848.000000,20848.000000,2.084800e+04,20848.000000,20848.000000,20848.000000,20848.0,20848.000000
mean,20.444887,3.454602e+01,0.792401,18.589298,-36.809603,19.048730,-31.719159,14.259371,7.707910e-01,2.878933,...,0.312113,15.755194,0.923637,66.679617,7.696212e+03,0.446309,0.092304,0.039632,0.0,1.053147
std,11.998162,3.049228e+04,1.097908,9.582665,9.021351,10.276899,10.350914,7.851465,1.823293e-01,0.797969,...,1.536693,9.531866,0.153723,81.413040,1.836728e+05,0.196391,0.054387,0.359958,0.0,0.224331
min,1.000000,-1.032700e+06,0.003000,0.000000,-55.270000,-0.806350,-55.379000,0.100790,6.123200e-17,1.000000,...,-3.140200,0.100790,0.008963,0.291040,-3.020100e+06,0.035294,0.022222,-1.570800,0.0,1.000000
25%,10.000000,7.867500e-01,0.139000,10.886000,-43.496000,10.382000,-38.604000,8.005900,6.687650e-01,2.000000,...,-0.091171,9.101025,0.926750,10.352750,-5.972250e+02,0.298040,0.044444,0.000000,0.0,1.000000
50%,21.000000,1.596000e+00,0.413000,19.840000,-38.100000,20.712000,-33.463000,13.248000,7.962000e-01,3.000000,...,0.848805,14.369000,0.975095,29.070500,0.000000e+00,0.470000,0.077778,0.000000,0.0,1.000000
75%,32.000000,3.727000e+00,0.994000,25.182000,-31.246000,26.610000,-25.803000,19.192750,9.170150e-01,3.000000,...,1.332350,20.456500,0.993030,116.152500,4.427250e+02,0.590000,0.133330,0.000000,0.0,1.000000
max,41.000000,1.031800e+06,22.315000,57.995000,-0.654080,57.995000,3.124600,53.861000,1.000000e+00,4.000000,...,3.141600,396.100000,1.000000,3903.500000,8.014900e+06,1.280000,0.500000,1.570800,0.0,2.000000


In [19]:
# preprocess extracted features
df = df.drop(['doc_id', 'phone_id'], axis=1) # drop values that aren't needed in classification
df = df.dropna()
nullsum = df.isnull().sum()
print(nullsum)

user_id                                           0
interstroke_time                                  0
stroke_duration                                   0
start_x                                           0
start_y                                           0
stop_x                                            0
stop_y                                            0
direct_end_to_end_distance                        0
mean_resultant_lenght                             0
orientation_flag                                  0
direction_of_end_to_end_line                      0
20%_pairwise_velocity                             0
50%_pairwise_velocity                             0
80%_pairwise_velocity                             0
20%_pairwise_acc                                  0
50%_pairwise_acc                                  0
80%_pairwise_acc                                  0
median_velocity_at_last_3_pts                     0
largest_deviation_from_end_to_end_line            0
20%_dev._fro

In [20]:
def clasify(data = None, target = None, kfolds = 5, neighbours_count = 5, random_state = 42):
    
    if (target is None or data is None):
        return -1
    
    kfold = StratifiedKFold(n_splits = kfolds, shuffle = True, random_state = random_state)
      
    target_users = data['user_id'].copy()
    target_users.loc[target_users != target] = 0
    target_users.loc[target_users == target] = 1
    
    # defining empty array for results
    predicts_array_svm     = []
    predicts_array_knn     = []
    far_array_svm     = []
    far_array_knn     = []
    frr_array_svm     = []
    frr_array_knn     = []
    
    kfold_iterrator = 0
    
    for train_index, test_index in kfold.split(data, target_users):
        
        print('user_id: ' + str(target) + ', kfold: ' + str(kfold_iterrator))
        kfold_iterrator = kfold_iterrator + 1
        
        #train test split input data for current user
        X_train_fold, X_test_fold = data.iloc[train_index], data.iloc[test_index]
        Y_train_fold, Y_test_fold = target_users.iloc[train_index], target_users.iloc[test_index]

        #drop user_id columns before training
        X_train_fold = X_train_fold.drop(['user_id'], axis=1)
        X_test_fold = X_test_fold.drop(['user_id'], axis=1)
                
        #scale value based on current train set
#         scaler = StandardScaler().fit(X_train_fold)  
#         X_train_fold = scaler.transform(X_train_fold)
#         X_test_fold = scaler.transform(X_test_fold)
                
        #train and test SVM
        svm_clf = svm.SVC(gamma='scale')
        svm_clf.fit(X_train_fold, Y_train_fold)  
        predict_svm = svm_clf.predict(X_test_fold)
        predicts_array_svm.append(accuracy_score(Y_test_fold, predict_svm))
        far_array_svm.append(metrics.fmr_score(Y_test_fold, predict_svm))
        frr_array_svm.append(metrics.fnmr_score(Y_test_fold, predict_svm))
        
        #train and test kNN
        knn_clf = KNeighborsClassifier(n_neighbors=neighbours_count, weights='distance')
        knn_clf.fit(X_train_fold, Y_train_fold)
        predict_knn = knn_clf.predict(X_test_fold)
        predicts_array_knn.append(accuracy_score(Y_test_fold, predict_knn))
        far_array_knn.append(metrics.fmr_score(Y_test_fold, predict_knn)) 
        frr_array_knn.append(metrics.fnmr_score(Y_test_fold, predict_knn))
    
        results = pd.DataFrame(
            np.array([[target, np.mean(predicts_array_svm), np.mean(predicts_array_knn), np.mean(far_array_svm), np.mean(far_array_knn), np.mean(frr_array_svm), np.mean(frr_array_knn)]]),
            columns=['user_id', 'accuracy_score_svm', 'accuracy_score_knn', 'far_svm', 'far_knn', 'frr_svm', 'frr_knn']
        )

    return results


In [21]:
user_ids = df.user_id.unique()
user_ids.sort()
user_ids

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41], dtype=int64)

In [10]:
predictions = pd.DataFrame()

for user_id in user_ids:
    print(user_id)
#     current_user = selectSelfUserData(df, 1)

    predictions = predictions.append(
        clasify(data = df.copy(), target = user_id, kfolds = 5, neighbours_count = 3)
    )

1
user_id: 1, kfold: 0
user_id: 1, kfold: 1
user_id: 1, kfold: 2
user_id: 1, kfold: 3
user_id: 1, kfold: 4
2
user_id: 2, kfold: 0
user_id: 2, kfold: 1
user_id: 2, kfold: 2
user_id: 2, kfold: 3
user_id: 2, kfold: 4
3
user_id: 3, kfold: 0
user_id: 3, kfold: 1
user_id: 3, kfold: 2
user_id: 3, kfold: 3
user_id: 3, kfold: 4
4
user_id: 4, kfold: 0
user_id: 4, kfold: 1
user_id: 4, kfold: 2
user_id: 4, kfold: 3
user_id: 4, kfold: 4
5
user_id: 5, kfold: 0
user_id: 5, kfold: 1
user_id: 5, kfold: 2
user_id: 5, kfold: 3
user_id: 5, kfold: 4
6
user_id: 6, kfold: 0
user_id: 6, kfold: 1
user_id: 6, kfold: 2
user_id: 6, kfold: 3
user_id: 6, kfold: 4
7
user_id: 7, kfold: 0
user_id: 7, kfold: 1
user_id: 7, kfold: 2
user_id: 7, kfold: 3
user_id: 7, kfold: 4
8
user_id: 8, kfold: 0
user_id: 8, kfold: 1
user_id: 8, kfold: 2
user_id: 8, kfold: 3
user_id: 8, kfold: 4
9
user_id: 9, kfold: 0
user_id: 9, kfold: 1
user_id: 9, kfold: 2
user_id: 9, kfold: 3
user_id: 9, kfold: 4
10
user_id: 10, kfold: 0
user_id: 10,

In [11]:
predictions

,user_id,accuracy_score_svm,accuracy_score_knn,far_svm,far_knn,frr_svm,frr_knn
0,1.0,0.981773,0.979279,0.000000,0.002883,1.000000,0.981579
0,2.0,0.941817,0.939467,0.000000,0.012580,1.000000,0.836728
0,3.0,0.963546,0.955247,0.000149,0.009557,1.000000,0.978860
0,4.0,0.989160,0.988152,0.000000,0.001067,1.000000,0.995556
0,5.0,0.980238,0.976640,0.000049,0.003866,1.000000,0.992712
0,6.0,0.973091,0.968438,0.000049,0.005126,1.000000,0.989286
0,7.0,0.971988,0.965800,0.000000,0.007698,1.000000,0.953714
0,8.0,0.970405,0.964505,0.000099,0.008452,1.000000,0.925203
0,9.0,0.978271,0.975585,0.000098,0.003285,1.000000,0.980098
0,10.0,0.981533,0.979662,0.000000,0.002297,1.000000,0.979221


In [12]:
# mean values of normal prediction
mean_predictions = predictions.copy()
mean_predictions = mean_predictions.mean()
mean_predictions

user_id               21.000000
accuracy_score_svm     0.975572
accuracy_score_knn     0.971481
far_svm                0.000070
far_knn                0.005534
frr_svm                0.999126
frr_knn                0.965775
dtype: float64

mean values of normal prediction

accuracy_score_svm     0.975572
accuracy_score_knn     0.971481
far_svm                0.000070
far_knn                0.005534
frr_svm                0.999126
frr_knn                0.965775

In [ ]:
# AIS NS algorithm
# 1. Construct set of detectors
# 2. Remove self patern detectors
# 3. 

In [ ]:
class detector:
    
    def __init__(self, value, feature_name)
        self.value = None
        self.
        self = False
    
    def check 

In [ ]:
# defining AIS Negative Selection algorithm
class neagtive_selection:
    
    def __init__(self, df = None self_user_id = None, features_batch_size = 100, threshold = 3, detectors_max_count = 100):
        
        if (df is None or self_user_id is None):
            return -1
        
        # split dataframe to self user data and non/self user data (impostor)
        self.data = df
        self.self_data, self.non_self_data = split_user_data(df = df, self_user_id = self_user_id)
        
        self.detectors = []
        self.detectors_max_count = detectors_max_count
        
        self.self_user_id = target_user
        self.features_batch_size = features_batch_size
        self.threshold = threshold
    
    def __get_boudaries(self):
        boundaries = self.data.description().loc[['min', 'max']].drop(['user_id'], axis=1)
        return boundaries
    
    def __detectors_generator(self):
        features_boundaries = self.__get_boudaries()
        
        detectors_current_count = 0
        while detectors_current_count < self.detectors_max_count:
            #generate detectors
        return None
    
    #     current_user = selectSelfUserData(df, 1)
    def fit(self):
        # 1. Construct set of detectors
        # 2. Remove self patern detectors
        return None
    
    def predict(self):
        return None


In [ ]:
# function to split data to self-user an non-self by user id
def split_user_data(df = None, self_user_id = None, user_id_column = 'user_id'):
    
    if (self_user_id is None or df is None):
        return 0
    
    user_df = df.copy()
    
    filter_userid = user_df[user_id_column] == self_user_id
    
    self_data = user_df[filter_userid].reset_index(drop=True)
    non_self_data = user_df[!filter_userid].reset_index(drop=True)
    
    return self_data, non_self_data

In [ ]:
# NS configuration
features_batch_size = 100
threshold = 3
detectors_count = 100

user_ns_dictionary = {}
for user_id in user_ids:
    print('Current user: ' + string(user_id))
        
    # create negative selection for current user, save it to dictionary
    user_ns_dictionary[user] = negative_selection(
        df = df.copy(), 
        self_user_id = user, 
        features_batch_size, 
        threshold, 
        detectors_count
    )    
    user_ns_dictionary[user].fit(self_data)
    user_ns_dictionary[user].predict(non_self_data)
    
    

In [36]:
for column in df:
    print(column)

user_id
interstroke_time
stroke_duration
start_x
start_y
stop_x
stop_y
direct_end_to_end_distance
mean_resultant_lenght
orientation_flag
direction_of_end_to_end_line
20%_pairwise_velocity
50%_pairwise_velocity
80%_pairwise_velocity
20%_pairwise_acc
50%_pairwise_acc
80%_pairwise_acc
median_velocity_at_last_3_pts
largest_deviation_from_end_to_end_line
20%_dev._from_end_to_end_line
50%_dev._from_end_to_end_line
80%_dev._from_end_to_end_line
average_direction
length_of_trajectory
ratio_end_to_end_dist_and_length_of_trajectory
average_velocity
median_acceleration_at_first_5_points
midstroke_pressure
midstroke_area_covered
midstroke_finger_orientation
change_of_finger_orientation
phone_orientation
